In [ ]:
import numpy as np
import sys
import time
#import multiprocessing as mp


radius = 1 # Hard-coded because it doesn't change.

In [ ]:
def user_input():
    
    while True:

        try:
#            dim = int(input("Please input a dimension: "))
            N_0 = int(input("Number of Monte-Carlo samples: "))
            K = int(input("And number of grid points K: "))
            
        except ValueError:
            print("Something went wrong, please try again.")
            continue
                
#        if dim < 2:
#            print("The number of dimensions must be positive and greater than 1.")
            # For 1D, python throws an error due to the array manipulation.
#            continue
                
        if N_0 < 0:
            print("The number of samples must be positive.")
            continue

        if K < 0:
            print("The number of samples must be positive.")
            continue
            
        else:
            # Success!
            break
        
    return [N_0, K] #[dim, N_0, K]

In [ ]:
# A function which calculates the volume of the big hypercube.

def hypercube(s_radius, dim):
    
    volume = (2*s_radius)**dim # For this project, s_radius = 1 
                               # but for generality I kept a variable.
    
    return volume

In [ ]:
# Used for determining chunk size for the Monte Carlo algorithm.

def mem_check_mc(N, dim):
    
    count = 1 # Keeping track.
    
    while True:
    
        try:
            #print("Trying: ", N)
            
            # Here we try to initialize an array and if that fails...
            some_array = np.empty([N, dim]) 
        
        except MemoryError:
            N = N/10 # ... we divide by 10 and try again.
            count *= 10 # How many 10's we've divided out.
            continue
            
        break
    
    # For memory-sake
    del some_array
    
    return [N, count]
    

In [ ]:
# Determines the variance^2 per the formula given
# in the confidence interval sheet.

def sample_variance(volumes):
    
    k_runs = volumes.size
    var_squared = 1/(k_runs-1)*np.sum((volumes-np.mean(volumes))**2)
    
    return var_squared

In [ ]:
# Vectorized Monte-Carlo. 

def vector_mc(s_radius, dim, N, seed):
    
    # Using the memory check to chunk out the samples.
    # Doesn't matter much in the final implementation with
    # the wrapper function but still playing it safe. 
    chunk_size, chunk_num = mem_check_mc(N, dim)
    
    inside_hyp = 0
    
    # Iterate over the chunks!
    for chunk in range(chunk_num):
        
        gen = np.random.RandomState(seed) # Seeding the distribution.
        
        while True: # Just in case. 
        
            try:
        
                # Generating the samples somewhere between -1 and 1
                samples = gen.uniform(-s_radius, s_radius, [chunk_size, dim])
            
            except MemoryError:
                print("Something went wrong! Try memory check again.")
                return 0
        
            break
            
        # Taking the norm of each coordinate with vectorized operations.
        dist = np.linalg.norm(samples, axis = 1)
        
        # Sorting so as to avoid checking every point.
        dist = np.sort(dist)
        
        # Determining where the samples no longer satisfy the radial condition.
        inside_hyp += np.where(dist <= 1)[-1][-1] + 1 # +1 to account for python array numbering starting at 0
    
    return inside_hyp/N # Returns the ratio.

In [ ]:
# Wraps the Monte-Carlo function to perform the variance estimate.

def mc_wrapper(radius, dims, N, rel_error):

    count = 1 # Start small.
    relative_error = 1 # Need a starting value for the while loop.

    volumes = np.array([]) # Initializing.
    
    vol_cube = hypercube(radius, dims) # Calculating volume of the big cube.

#     total_start = time.time() # Previously used for timing.

    # Begin while loop with high count as a stopping condition.
    # Will stop when either the count is too high or the relative
    # error passes below the desired threshhold.
    
    while (relative_error > rel_error) and (count < 100000):

        seed = time.time() # Seeding with time and count in case it runs too fast.

        # Appending each sample set to an array to make calculating the mean easy.
        volumes = np.append(volumes, vol_cube*vector_mc(radius, dims, N, int(seed+count)))

        # Calculating the variances and relative error
        if count >= 2:
            variance = sample_variance(volumes) # Actually a squared value.
            std = np.sqrt(variance/count) # Standard Deviation.

            relative_error = 2*std/np.mean(volumes) # 95% confidence
    
        count += 1
        
    total_end = time.time()
    #print("Total Monte Carlo time: ", total_end - total_start, "s")

    return [np.mean(volumes), relative_error]

In [ ]:
# This code performs the hypercube integration using 
# linear algebraic/vectorized techniques. 

def cube_int_linalg(s_radius, K, dim):

    start = time.time()
    
    # Memory error catching loop of try/excepts.
    while True:
        
        num_cubes = K**dim # May change with memory errors.
        grids = np.empty([dim,K]) # K points!
        
        try: # Initialize big empty arrays.
            
            #print("Trying K = ", K)
            coords = np.empty([dim,num_cubes])
            coord_dist = np.empty([num_cubes])
            #break
            
        except MemoryError as err:
        
            K = int(K/2) # Try and try again.
            
#            print("Sorry! K is too high. Cannot compute the volume of the sphere to the requested precision.")
#            print("Attempting a computation with fewer K... \n")
#            print("New K: ", K)

            continue
    
        vol_cube = (2/K)**dim # Now that K is set, we can calculate the volume
                              # of each little cube.
    
        h = s_radius/K # Separation h.
        norm_h = h*np.sqrt(dim) # Extent of h in n-dimensions.
        
        try: # To be elucidated...
    
            # Generating the grid in every dimension
            for i in range(dim):
                grids[i,] = np.linspace(0, s_radius-h, K)
    
            # This is the expensive part. With K^d coordinate points, this 
            # rapidly blows-up. Having the extra options set to False in meshgrid
            # helps but unfortunately the operation of putting everything together
            # is what takes up the most memory. 
            coords = np.array(np.meshgrid(*grids, sparse=False, copy=False)).T.reshape(-1,dim)
        
            # Calculate the distance to the 'starting' corner of the cubes.
            # We add the norm of h first for inside/outside logic below.
            coord_dist = np.linalg.norm(coords, axis = -1) + norm_h
            
            del coords # Saving memory! 
        
            coord_dist = np.sort(coord_dist) # For checking inside/outside efficiently.
            
            break # Out of the while True loop.
        
        except MemoryError as err: # If the grid generation is too hefty.
    
            K = K - 20 # Since I was able to initalize the arrays in the first
                       # place we're probably pretty close.
            
            continue
    
    end = time.time()
    
    coords_time = end-start
    
    inside = 0
    outside = 0
    between = 0   
    
    start = time.time()
    
    # **************** HARD-CODING PHI ********************
    
    # For generic phi functions see the commented out block below
    # Using sort is faster for such large arrays so I've left this code in.
    # Using np.where is essentially a vectorized phi anyway! 
    
    # Since we sorted, we can find the index at which the furthest point of the cube is 
    # at or inside of the circle radius and that's the number of cubes inside.
    inside = np.where(coord_dist <= s_radius)[-1][-1]
    
    coord_dist = coord_dist - norm_h
    
    # For the outside, we can find the index at which the closest point of the cube is
    # at or outside the circle radius and subtract that number from the total number of cubes.
    outside = num_cubes - np.where(coord_dist >= s_radius)[-1][0]
    
    # They all sum to the number of cubes so we simply use that to find the number between.
    between = num_cubes - inside - outside
        
    # **************** FOR GENERIC PHI ********************
    
    # Check each point. If the furthest point of the cube is inside then we know
    # that cube is definitely inside
    
    #for i in range(len(coord_dist)):
    #    if phi(coord_dist[i] + norm_h):                    # < s_radius:            
    #        inside += 1
              
    # If the furthest edge is out but the closest point is in, we know the cube is between.
    
    #    elif phi(coord_dist[i]):                    # < s_radius:
    #        between += 1
           
    # If neither then the cube is definitely outside. 
    
    #    else:
    #        outside += 1
            
    # *****************************************************
    
    end = time.time()
    
    check_time = end-start
    
    if between < 0:
        print("Something went wrong! The number of little cubes is incorrect.")
        #print(num_cubes)
        #print(inside)
        #print(outside)
        return 0, 0
    
    low_bound = inside*vol_cube # The hypersphere definitely can't be smaller than that.
    up_bound = (inside+between)*vol_cube # Or larger than this.
    
    #print("Coordinate Generation", coords_time)
    #print("Checking Each Coord", check_time)
    
    return low_bound, up_bound, K

In [ ]:
# PROBLEM 1
N_0 = 100000
N_cube = 1000
rerror = 0.0001

for i in range(3):

    dims = i+2 # Iterating over the dimensions.
    start = time.time()
    mc_vol, mc_err = mc_wrapper(radius, dims, N_0, rerror)
    end = time.time()
    mc_time = end-start
    print("In", dims, "dimensions,")
    print("Monte Carlo Volume and Relative Error", mc_vol, "+/-", mc_err)
    print("Monte Carlo Time", mc_time, "\n")
    
    start = time.time()
    low, up, K_final  = cube_int_linalg(radius, N_cube, dims)
    end = time.time()
    cube_time = end-start
    
    int_vol = low + (up-low)/2
    print("Cube Integration Volume Bounds", low, up)
    print("Cube Integration Time", cube_time, "with K:", K_final, "\n")

In [ ]:
# PROBLEM 2
N_0 = 1000000
rerror = 0.0001

for i in range(4):

    dims = i+2
    mc_vol, mc_err = mc_wrapper(radius, dims, N_0, rerror)
    print("In", dims, "dimensions,")
    print("Monte Carlo Volume and Relative Error", mc_vol, "+/-", mc_err)
    
    low, up, _ = cube_int_linalg(radius, int(N_0**(1/dims)), dims)
    int_vol = low + (up-low)/2
    print("Cube Integration Volume", int_vol, "\n")

In [ ]:
# PROBLEM 3
N_0, K = user_input()
print()
#mc_2, mc_2_err = mc_wrapper(radius, 2, N_0, rerror)
#mc_3, mc_3_err = mc_wrapper(radius, 3, N_0, rerror)
print("Using Monte-Carlo in 2D we get pi to be:", mc_2, "+/-", mc_2_err)
print("And in 3D we get pi to be:", 3*mc_3/4, "+/-", mc_3_err)
print()
    
low, up, _ = cube_int_linalg(radius, K, 2)
int_vol = low + (up-low)/2
print("Cube Integration pi:", int_vol, "\n")


In [ ]:
def phi(point, circle_radius):
    
    if point <= circle_radius:
        return True
    else:
        return False